In [26]:
%load_ext autoreload
%autoreload 2

In [113]:
from MagNumEvaluator import MagNumEvaluator
from tqdm import tqdm_notebook
import pickle
import pandas as pd

In [99]:
from mag_num_utils import detect_number

In [100]:
nc = pickle.load(open('../../data/wikipedia/preprocess0.05Bnotable/NumeralAsNumeral/nc.dat','rb'))
idx2word = pickle.load(open('../../data/wikipedia/preprocess0.05Bnotable/NumeralAsNumeral/idx2word.dat','rb'))
wc = pickle.load(open('../../data/wikipedia/preprocess0.05Bnotable/NumeralAsNumeral/wc.dat','rb'))
word2idx = pickle.load(open('../../data/wikipedia/preprocess0.05Bnotable/NumeralAsNumeral/word2idx.dat','rb'))

In [155]:
out_dicts = pickle.load(open('data/numerals_dicts_num_0.05B.pkl', 'rb'))

In [157]:
evaluator = MagNumEvaluator(numerals_dict=out_dicts, type='NUM')

In [158]:
res_num = {}

In [173]:
pd.DataFrame(res_num).transpose()

,0,1,2,3
p-50,0.032609,0.054348,0.467391,0.945652
p-100,0.021739,0.021739,0.456522,0.967391
p-200,0.021739,0.032609,0.565217,0.978261
p-300,0.010870,0.021739,0.532609,0.978261
p-500,0.032609,0.076087,0.554348,0.978261
p-log-50,0.010870,0.032609,0.489130,0.967391
p-log-100,0.000000,0.021739,0.532609,0.967391
p-log-200,0.021739,0.043478,0.554348,0.978261
p-log-300,0.043478,0.043478,0.543478,0.978261
p-log-500,0.000000,0.032609,0.565217,0.967391


In [160]:
print('=== Evaluating Prototype ===')
prototype_size = ['50', '100', '200', '300', '500']
for sz in prototype_size:
    trained_prototypes = pickle.load(
        open('../../data/wikipedia/save/0.05Bnotable/prototypes/{}/trained_prototypes_epoch5.dat'.format(sz),'rb')) 
    idx2vec_i = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/prototypes/{}/idx2vec_i_epoch5.dat'.format(sz),'rb'))
    evaluator.load_prototype(trained_prototypes, idx2vec_i=idx2vec_i)
    OVA, OVAR, SC, BC = evaluator.evaluate_all()
    res_num['p-{}'.format(sz)] = [OVA, OVAR, SC, BC]

=== Evaluating Prototype ===


In [161]:
print('=== Evaluating Prototype LOG ===')
prototype_size = ['50', '100', '200', '300', '500']
for sz in prototype_size:
    trained_prototypes = pickle.load(
        open('../../data/wikipedia/save/0.05Bnotable/prototypes_log/{}/trained_prototypes_epoch5_{}_1.0.dat'.format(sz, sz),'rb')) 
    idx2vec_i = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/prototypes_log/{}/idx2vec_i_epoch5.dat'.format(sz),'rb'))
    evaluator.load_prototype(trained_prototypes, idx2vec_i=idx2vec_i, log_space=True)
    OVA, OVAR, SC, BC = evaluator.evaluate_all()
    res_num['p-log-{}'.format(sz)] = [OVA, OVAR, SC, BC]

=== Evaluating Prototype LOG ===


In [162]:
print('=== Evaluating GMM-RD-SOFT ===')
gmms = ['50', '100', '200', '300', '500']
for gmm_name in gmms:
    trained_prototypes = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/gmm/{}-rd/trained_gmms_epoch5_{}.dat'.format(gmm_name, gmm_name),'rb'))
    gmm = pickle.load(open('../../data/wikipedia/preprocess0.05Bnotable/NumeralAsNumeral/gmm/gmm-{}-rd.dat'.format(gmm_name),'rb'))
    idx2vec_i = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/gmm/{}-rd/idx2vec_i_epoch5.dat'.format(gmm_name),'rb'))
    idx2vec_o = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/gmm/{}-rd/idx2vec_i_epoch5.dat'.format(gmm_name),'rb'))
    evaluator.load_GMM(trained_prototypes, gmm, idx2vec_i=idx2vec_i, idx2vec_o=idx2vec_o)
    OVA, OVAR, SC, BC = evaluator.evaluate_all()
    res_num['gmm-rd-soft-{}'.format(gmm_name)] = [OVA, OVAR, SC, BC]

=== Evaluating GMM-RD-SOFT ===


In [163]:
print('=== Evaluating GMM-RD-SOFT-LOG ===')
gmms = ['50', '100', '200', '300', '500']
for gmm_name in gmms:
    trained_prototypes = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/gmm_log/{}-rd-soft/trained_gmms_epoch5_{}.dat'.format(gmm_name, gmm_name),'rb'))
    gmm = pickle.load(open('../../data/wikipedia/preprocess0.05Bnotable/NumeralAsNumeral/gmm_log/gmm-{}-rd-soft.dat'.format(gmm_name),'rb'))
    idx2vec_i = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/gmm_log/{}-rd-soft/idx2vec_i_epoch5.dat'.format(gmm_name),'rb'))
    idx2vec_o = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/gmm_log/{}-rd-soft/idx2vec_o_epoch5.dat'.format(gmm_name),'rb'))
    evaluator.load_GMM(trained_prototypes, gmm, idx2vec_i=idx2vec_i, idx2vec_o=idx2vec_o, log_space=True)
    OVA, OVAR, SC, BC = evaluator.evaluate_all()
    res_num['gmm-log-rd-soft-{}'.format(gmm_name)] = [OVA, OVAR, SC, BC]

=== Evaluating GMM-RD-SOFT-LOG ===


In [164]:
print('=== Evaluating GMM-RD-HARD ===')
gmms = ['50', '100', '200', '300', '500']
for gmm_name in gmms:
    trained_prototypes = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/gmm/{}-rd-hard/trained_gmms_epoch5_{}.dat'.format(gmm_name, gmm_name),'rb'))
    gmm = pickle.load(open('../../data/wikipedia/preprocess0.05Bnotable/NumeralAsNumeral/gmm/gmm-{}-rd-hard.dat'.format(gmm_name),'rb'))
    idx2vec_i = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/gmm/{}-rd-hard/idx2vec_i_epoch5.dat'.format(gmm_name),'rb'))
    idx2vec_o = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/gmm/{}-rd-hard/idx2vec_i_epoch5.dat'.format(gmm_name),'rb'))
    evaluator.load_GMM(trained_prototypes, gmm, idx2vec_i=idx2vec_i, idx2vec_o=idx2vec_o)
    OVA, OVAR, SC, BC = evaluator.evaluate_all()
    res_num['gmm-rd-hard-{}'.format(gmm_name)] = [OVA, OVAR, SC, BC]

=== Evaluating GMM-RD-HARD ===


In [165]:
print('=== Evaluating GMM-RD-HARD-LOG ===')
gmms = ['50', '100', '200', '300', '500']
for gmm_name in gmms:
    trained_prototypes = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/gmm_log/{}-rd-hard/trained_gmms_epoch5_{}.dat'.format(gmm_name, gmm_name),'rb'))
    gmm = pickle.load(open('../../data/wikipedia/preprocess0.05Bnotable/NumeralAsNumeral/gmm_log/gmm-{}-rd-hard.dat'.format(gmm_name),'rb'))
    idx2vec_i = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/gmm_log/{}-rd-hard/idx2vec_i_epoch5.dat'.format(gmm_name),'rb'))
    idx2vec_o = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/gmm_log/{}-rd-hard/idx2vec_o_epoch5.dat'.format(gmm_name),'rb'))
    evaluator.load_GMM(trained_prototypes, gmm, idx2vec_i=idx2vec_i, idx2vec_o=idx2vec_o, log_space=True)
    OVA, OVAR, SC, BC = evaluator.evaluate_all()
    res_num['gmm-log-rd-hard-{}'.format(gmm_name)] = [OVA, OVAR, SC, BC]

=== Evaluating GMM-RD-HARD-LOG ===


In [166]:
idx2vec_o = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/1/LSTM/idx2vec_o_epoch5.dat','rb'))
idx2vec_i = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/1/LSTM/idx2vec_i_epoch5.dat','rb'))
LSTM_model_path = '../../data/wikipedia/save/0.05Bnotable/1/LSTM/sgns_epoch5.pt'
evaluator.load_LSTM(idx2vec_i, idx2vec_o, LSTM_model_path)
OVA, OVAR, SC, BC = evaluator.evaluate_all()
res_num['LSTM'] = [OVA, OVAR, SC, BC]

In [168]:
print('=== Evaluating Fixed Baseline ===')
idx2vec_i = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/fixed/idx2vec_i_epoch5.dat','rb'))
evaluator.load_fixed(idx2vec_i=idx2vec_i)
OVA, OVAR, SC, BC = evaluator.evaluate_all()
res_num['FIXED'] = [OVA, OVAR, SC, BC]

=== Evaluating Fixed Baseline ===


In [170]:
nc = pickle.load(open('../../data/wikipedia/preprocess0.05Bnotable/NumeralAsTokenUnkNumeral/nc.dat','rb'))
idx2word_nat = pickle.load(open('../../data/wikipedia/preprocess0.05Bnotable/NumeralAsTokenUnkNumeral/idx2word.dat','rb'))
wc = pickle.load(open('../../data/wikipedia/preprocess0.05Bnotable/NumeralAsTokenUnkNumeral/wc.dat','rb'))
word2idx_nat = pickle.load(open('../../data/wikipedia/preprocess0.05Bnotable/NumeralAsTokenUnkNumeral/word2idx.dat','rb'))
out_dicts_nat = {}
for k, v in out_dicts.items():
    word = idx2word[v]
    idx_nat = word2idx_nat[word]
    out_dicts_nat[k] = idx_nat

In [172]:
evaluator.reload(numerals_dict=out_dicts_nat, idx2word=idx2word_nat, word2idx=word2idx_nat, type='NUM')

idx2vec_o = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/1/NumeralAsTokenUnkNumeral/idx2vec_o_epoch5.dat','rb'))
idx2vec_i = pickle.load(open('../../data/wikipedia/save/0.05Bnotable/1/NumeralAsTokenUnkNumeral/idx2vec_i_epoch5.dat','rb'))

evaluator.load_TOKEN(idx2vec_i=idx2vec_i, idx2vec_o=idx2vec_o)
OVA, OVAR, SC, BC = evaluator.evaluate_all()
res_num['TOKEN'] = [OVA, OVAR, SC, BC]